# Lesson 12 - 01_matmul.ipynb: 从零实现矩阵乘法 — 逐步优化之旅

本节课以 MNIST 数据集为载体，从最朴素的三重循环开始，逐步优化矩阵乘法的实现，最终达到约 **500万倍** 的加速。
整个过程串联了 Python 基础、PyTorch tensor 操作、broadcasting 机制、Einstein 求和以及 CUDA 编程等核心概念。

## 1. 获取数据 & 基础工具

- 下载 MNIST 手写数字数据集（`urlretrieve` + `gzip` + `pickle`）
- 用纯 Python 的 `chunks` 生成器和 `itertools.islice` 将一维像素列表重塑为 28×28 图像
- 手写一个简单的 `Matrix` 类，支持 `m[i, j]` 二维索引，体会 tensor 的本质

## 2. Matrix & Tensor

- 将 numpy 数组转为 PyTorch `tensor`
- `x_train.reshape((-1, 28, 28))` 得到图像张量
- 理解 shape、type 等基本属性

## 3. 随机数生成器

- 手写 **Wichmann-Hill** 伪随机数生成算法（Python 2.3 之前使用的算法）
- 用 `os.fork()` 演示子进程继承随机状态导致的安全隐患（父子进程产生相同随机数）
- 对比纯 Python 随机数 vs `torch.randn` 的速度差异

## 4. 矩阵乘法的逐步优化

这是本课的核心——用多种方法实现 matmul，每一步都带来数量级的加速：

### v0: 纯 Python 三重循环（~500ms）
```python
for i in range(ar):
    for j in range(bc):
        for k in range(ac):
            c[i,j] += a[i,k] * b[k,j]
```

### v1: Numba JIT 加速内层循环
- 用 `@njit` 编译 `dot` 函数，消除最内层 Python 循环
- 仅剩两层 Python 循环

### v2: 逐元素运算（Elementwise ops）
- 利用 PyTorch 的向量化运算消除最内层循环：`c[i,j] = (a[i,:] * b[:,j]).sum()`
- 或直接用 `torch.dot(a[i,:], b[:,j])`
- 顺带介绍了 Frobenius 范数

### v3: Broadcasting 版本（<0.1ms，~5000x 加速）
- 利用 broadcasting 一次计算整行：`c[i] = (a[i,:,None] * b).sum(dim=0)`
- 从三重循环减少到仅一层循环

## 5. Broadcasting 详解

这是理解上述优化的关键概念：

- **标量广播**: `a > 0`，标量 0 被广播到与 a 相同的维度
- **向量→矩阵广播**: `m + c`，向量 c 沿缺失维度广播
- **`expand_as` & stride**: 广播不真正复制数据，而是将 stride 设为 0
- **`unsqueeze` / `None` 索引**: 在指定位置插入维度 1
- **广播规则**: 从尾部维度开始对齐，维度相等或其中一个为 1 即兼容

## 6. Einstein 求和 (einsum)

- 用紧凑的下标记法表达乘积与求和：`torch.einsum('ik,kj->ij', a, b)`
- 重复的字母（k）表示沿该轴相乘，输出中省略的字母表示沿该轴求和
- 可以拆解为中间步骤理解：先 `'ik,kj->ikj'`（逐元素乘），再 `.sum(1)`（沿 k 求和）

## 7. PyTorch 内置运算

- 直接使用 `torch.matmul` 或 `@` 运算符
- 底层调用高度优化的 BLAS 库

## 8. CUDA 编程入门

- 先用 Python 模拟 GPU kernel 的执行模型（grid + kernel 函数）
- `launch_kernel` 模拟 GPU 的网格调度
- 然后使用 `numba.cuda` 的 `@cuda.jit` 编写真正的 CUDA kernel
- 关键概念：`cuda.grid(2)` 获取线程坐标、`blockspergrid`、`TPB`(threads per block)
- 最终对比：`x_train.cuda() @ weights.cuda()` 利用 PyTorch 的 CUDA 后端

**最终加速**：从纯 Python 的 ~500ms 到 CUDA 的 ~0.5ms，总计约 **500万倍** 提升！

## 总结：优化路径一览

| 版本 | 方法 | 关键技术 |
|------|------|----------|
| v0 | 三重 Python 循环 | 纯 Python |
| v1 | Numba JIT dot | `@njit` 编译内层循环 |
| v2 | 逐元素向量化 | `(a[i,:] * b[:,j]).sum()` / `torch.dot` |
| v3 | Broadcasting | `(a[i,:,None] * b).sum(dim=0)` |
| v4 | Einstein 求和 | `torch.einsum('ik,kj->ij', a, b)` |
| v5 | PyTorch 内置 | `a @ b` / `torch.matmul` |
| v6 | CUDA kernel | `numba.cuda` / PyTorch CUDA |